In [1]:
import os
import sys
import json
import ipaddress
import numpy as np
from torch import nn
from sklearn.model_selection import train_test_split

class Resource:
  def __init__(self, RequestID, Site, LoadURL, LoadDomain, 
      Type, MimeType, RemoteIPAddr, ModTime):
    self.request_id = RequestID
    self.site = Site
    self.load_url = LoadURL
    self.load_domain = LoadDomain
    self.resource_type = Type
    self.mimetype = MimeType
    self.ip_addr = RemoteIPAddr
    self.mod_time = ModTime

# Function to parse and create Resources from JSON
def extract_from_JSON(filename):
    ip_to_occurrences = dict()
    ip_to_sites = dict()
    site_date_ipset = dict()
    with open(filename) as f:
        for line in f:
            data = json.loads(line)
            resource = Resource(data["RequestID"], data["Site"], data["LoadURL"], 
                                data["LoadDomain"], data["Type"], data["MimeType"], 
                                data["RemoteIPAddr"], data["ModTime"])
            
            # For ip occurrence filtering in entire dataset
            ip_to_occurrences.setdefault(resource.ip_addr, set())
            occurrence_event = resource.mod_time + " " + resource.site
            ip_to_occurrences[resource.ip_addr].add(occurrence_event)
    
            # For ips unique to a website
            ip_to_sites.setdefault(resource.ip_addr, set())
            ip_to_sites[resource.ip_addr].add(resource.site)
            
            # extracting samples
            site_date_ipset.setdefault(resource.site, dict())
            site_date_ipset[resource.site].setdefault(resource.mod_time, set())
            site_date_ipset[resource.site][resource.mod_time].add(resource.ip_addr)
    return ip_to_occurrences, ip_to_sites, site_date_ipset

In [3]:
filename = "../output/02-06-2020_1000-recurring_output.json"
ip_to_occurrences, ip_to_sites, site_date_ipset = extract_from_JSON(filename)

In [4]:
print(len(site_date_ipset), len(ip_to_occurrences), len(ip_to_sites))

974 54622 54622


In [5]:
# filtering passes: 
ips_to_remove = set()
for ip in ip_to_occurrences:
    occurrences = ip_to_occurrences[ip]
    if len(occurrences) < 20:
       ips_to_remove.add(ip)

for ip in ips_to_remove:
    try:
        del ip_to_occurrences[ip]
        del ip_to_sites[ip]
    except KeyError:
        print(ip, " Not Present")
     
unique_ip_to_site = dict()
# Does it make sense to filter the site out?
for ip in ip_to_sites:
    if len(ip_to_sites[ip]) == 1:
        site = ip_to_sites[ip].pop()
        unique_ip_to_site[ip] = site
        try:
            del site_date_ipset[site]
        except KeyError:
            print(site, " Not Present in map")
        try:
            del ip_to_occurrences[ip]
        except KeyError:
            print(ip, " Not Present")

# ip_to_occurrences holds all ips after filtering

104.com.tw  Not Present in map
104.com.tw  Not Present in map
104.com.tw  Not Present in map
104.com.tw  Not Present in map
11st.co.kr  Not Present in map
11st.co.kr  Not Present in map
11st.co.kr  Not Present in map
11st.co.kr  Not Present in map
11st.co.kr  Not Present in map
11st.co.kr  Not Present in map
11st.co.kr  Not Present in map
11st.co.kr  Not Present in map
11st.co.kr  Not Present in map
11st.co.kr  Not Present in map
123rf.com  Not Present in map
123rf.com  Not Present in map
123rf.com  Not Present in map
123rf.com  Not Present in map
123rf.com  Not Present in map
126.com  Not Present in map
126.com  Not Present in map
126.com  Not Present in map
126.com  Not Present in map
126.com  Not Present in map
126.com  Not Present in map
126.com  Not Present in map
126.com  Not Present in map
1337x.to  Not Present in map
1688.com  Not Present in map
1688.com  Not Present in map
1688.com  Not Present in map
1688.com  Not Present in map
1688.com  Not Present in map
1688.com  Not Pres

china.com.cn  Not Present in map
chinadaily.com.cn  Not Present in map
chinadaily.com.cn  Not Present in map
chinadaily.com.cn  Not Present in map
chinadaily.com.cn  Not Present in map
chip.de  Not Present in map
chip.de  Not Present in map
chip.de  Not Present in map
chip.de  Not Present in map
chip.de  Not Present in map
chip.de  Not Present in map
chip.de  Not Present in map
chip.de  Not Present in map
chip.de  Not Present in map
chip.de  Not Present in map
chiphell.com  Not Present in map
chiphell.com  Not Present in map
chiphell.com  Not Present in map
chiphell.com  Not Present in map
chiphell.com  Not Present in map
chiphell.com  Not Present in map
chouftv.ma  Not Present in map
chron.com  Not Present in map
chron.com  Not Present in map
chron.com  Not Present in map
chron.com  Not Present in map
chron.com  Not Present in map
chron.com  Not Present in map
cimaclub.com  Not Present in map
cinecalidad.to  Not Present in map
cinecalidad.to  Not Present in map
cinecalidad.to  Not Pre

gazeta.ru  Not Present in map
gazeta.ru  Not Present in map
gazeta.ru  Not Present in map
gazeta.ru  Not Present in map
gazeta.ru  Not Present in map
gazeta.ru  Not Present in map
gazeta.ru  Not Present in map
gazeta.ru  Not Present in map
gazeta.ru  Not Present in map
gazeta.ru  Not Present in map
gazeta.ru  Not Present in map
gazetaexpress.com  Not Present in map
gazetaexpress.com  Not Present in map
gazetaexpress.com  Not Present in map
gazetaexpress.com  Not Present in map
gazetaexpress.com  Not Present in map
gazetaexpress.com  Not Present in map
gazetaexpress.com  Not Present in map
gazetaexpress.com  Not Present in map
gazetaexpress.com  Not Present in map
gazetaexpress.com  Not Present in map
gazetaexpress.com  Not Present in map
gazetaexpress.com  Not Present in map
gazetaexpress.com  Not Present in map
gearbest.com  Not Present in map
gearbest.com  Not Present in map
gearbest.com  Not Present in map
gearbest.com  Not Present in map
gearbest.com  Not Present in map
gearbest.co

ibm.com  Not Present in map
ibm.com  Not Present in map
ibm.com  Not Present in map
ibm.com  Not Present in map
ibm.com  Not Present in map
ibm.com  Not Present in map
ibm.com  Not Present in map
ibm.com  Not Present in map
ibm.com  Not Present in map
icicibank.com  Not Present in map
icicibank.com  Not Present in map
icloud.com  Not Present in map
icloud.com  Not Present in map
icloud.com  Not Present in map
icloud.com  Not Present in map
icloud.com  Not Present in map
idntimes.com  Not Present in map
idntimes.com  Not Present in map
idntimes.com  Not Present in map
ieee.org  Not Present in map
ifeng.com  Not Present in map
ifeng.com  Not Present in map
ifeng.com  Not Present in map
ifeng.com  Not Present in map
ifeng.com  Not Present in map
ifeng.com  Not Present in map
ifeng.com  Not Present in map
ifeng.com  Not Present in map
ifeng.com  Not Present in map
ifeng.com  Not Present in map
ifeng.com  Not Present in map
ign.com  Not Present in map
ilovepdf.com  Not Present in map
imdb.c

marriott.com  Not Present in map
marriott.com  Not Present in map
marriott.com  Not Present in map
mashable.com  Not Present in map
mathrubhumi.com  Not Present in map
mathworks.com  Not Present in map
mathworks.com  Not Present in map
mathworks.com  Not Present in map
mathworks.com  Not Present in map
mathworks.com  Not Present in map
mawdoo3.com  Not Present in map
mawdoo3.com  Not Present in map
mawdoo3.com  Not Present in map
mawdoo3.com  Not Present in map
mayoclinic.org  Not Present in map
mayoclinic.org  Not Present in map
mayoclinic.org  Not Present in map
mediafire.com  Not Present in map
mediafire.com  Not Present in map
medium.com  Not Present in map
medium.com  Not Present in map
medium.com  Not Present in map
medium.com  Not Present in map
medium.com  Not Present in map
medium.com  Not Present in map
mega.nz  Not Present in map
mega.nz  Not Present in map
mega.nz  Not Present in map
mega.nz  Not Present in map
mega.nz  Not Present in map
mega.nz  Not Present in map
mega.nz

ozon.ru  Not Present in map
ozon.ru  Not Present in map
ozon.ru  Not Present in map
ozon.ru  Not Present in map
ozon.ru  Not Present in map
ozon.ru  Not Present in map
ozon.ru  Not Present in map
ozon.ru  Not Present in map
ozon.ru  Not Present in map
ozon.ru  Not Present in map
ozon.ru  Not Present in map
ozon.ru  Not Present in map
ozon.ru  Not Present in map
ozon.ru  Not Present in map
ozon.ru  Not Present in map
ozon.ru  Not Present in map
ozon.ru  Not Present in map
ozon.ru  Not Present in map
ozon.ru  Not Present in map
ozon.ru  Not Present in map
padlet.com  Not Present in map
padlet.com  Not Present in map
padlet.com  Not Present in map
pandora.com  Not Present in map
pantip.com  Not Present in map
pantip.com  Not Present in map
pantip.com  Not Present in map
pantip.com  Not Present in map
patch.com  Not Present in map
patch.com  Not Present in map
patch.com  Not Present in map
patreon.com  Not Present in map
patria.org.ve  Not Present in map
patria.org.ve  Not Present in map
p

tarafdari.com  Not Present in map
tarafdari.com  Not Present in map
tarafdari.com  Not Present in map
tarafdari.com  Not Present in map
tarafdari.com  Not Present in map
target.com  Not Present in map
td.com  Not Present in map
td.com  Not Present in map
td.com  Not Present in map
td.com  Not Present in map
td.com  Not Present in map
td.com  Not Present in map
td.com  Not Present in map
td.com  Not Present in map
teacherspayteachers.com  Not Present in map
teamviewer.com  Not Present in map
teamviewer.com  Not Present in map
teamviewer.com  Not Present in map
teamviewer.com  Not Present in map
teamviewer.com  Not Present in map
teamviewer.com  Not Present in map
teamviewer.com  Not Present in map
techradar.com  Not Present in map
telegraph.co.uk  Not Present in map
telegraph.co.uk  Not Present in map
telegraph.co.uk  Not Present in map
telegraph.co.uk  Not Present in map
telegraph.co.uk  Not Present in map
telegraph.co.uk  Not Present in map
telegraph.co.uk  Not Present in map
telegrap

vice.com  Not Present in map
vidio.com  Not Present in map
vimeo.com  Not Present in map
vimeo.com  Not Present in map
visualstudio.com  Not Present in map
visualstudio.com  Not Present in map
viva.co.id  Not Present in map
viva.co.id  Not Present in map
viva.co.id  Not Present in map
viva.co.id  Not Present in map
viva.co.id  Not Present in map
vnexpress.net  Not Present in map
vnexpress.net  Not Present in map
vnexpress.net  Not Present in map
vnexpress.net  Not Present in map
vnexpress.net  Not Present in map
vnexpress.net  Not Present in map
vnexpress.net  Not Present in map
vnexpress.net  Not Present in map
vnexpress.net  Not Present in map
vnexpress.net  Not Present in map
vnexpress.net  Not Present in map
vnexpress.net  Not Present in map
vnexpress.net  Not Present in map
vnexpress.net  Not Present in map
vnexpress.net  Not Present in map
vnexpress.net  Not Present in map
vnexpress.net  Not Present in map
vnexpress.net  Not Present in map
vnexpress.net  Not Present in map
vnexpr

In [6]:
print(len(site_date_ipset))

193


In [7]:
# create site to encoded ipset mapping
labels = []
encodings = []
all_ips = list(ip_to_occurrences.keys())
print(len(all_ips))
for site in site_date_ipset:
    date_to_ipset = site_date_ipset[site]
    for date in date_to_ipset:
        ipset = date_to_ipset[date]
        iparray = [0] * len(all_ips)
        for ip in ipset:
            if ip in ip_to_occurrences:
                iparray[all_ips.index(ip)] = 1
        labels.append(site)
        encodings.append(np.asarray(iparray))

print(len(labels), len(encodings))
                

9760
20786 20786


In [8]:
import pyasn
asndb = pyasn.pyasn('pyasn')
asn_set = set()
ip_to_asn = dict()
for ip in ip_to_occurrences:
    try:
        asn,_= asndb.lookup(ip)
        ip_to_asn[ip] = asn
        asn_set.add(asn)
    except:
        print(ip, " not mapped to asn")
        continue

  not mapped to asn


In [9]:
sites_asn = []
asn_enc = []
asn_list = list(asn_set)
for site in site_date_ipset:
    date_to_ipset = site_date_ipset[site]
    for date in date_to_ipset:
        ipset = date_to_ipset[date]
        asn_array = [0]*len(asn_list)
        for ip in ipset:
            if ip in ip_to_occurrences:
                try:
                    asn_array[asn_list.index(ip_to_asn[ip])] = 1
                except Exception as e:
                    continue
        sites_asn.append(site)
        asn_enc.append(np.asarray(asn_array))

In [10]:
print(len(sites_asn[0]), len(asn_enc[0]))
print(len(labels[0]), len(encodings[0]))

1 346
1 9760


In [34]:
import torch.nn.functional as F
class NNet(nn.Module):
    def __init__(self, input_size, output_size):
        super(NNet, self).__init__()
        self.fc1 = nn.Linear(input_size, 400)
        self.fc2 = nn.Linear(400, 100)
        self.fc3 = nn.Linear(100, output_size)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        return self.fc3(x)

In [35]:
from sklearn import preprocessing
import torch

tensor_encodings = torch.Tensor(encodings)
le_labels = preprocessing.LabelEncoder()
tensor_labels = le_labels.fit_transform(labels)
tensor_labels = torch.as_tensor(tensor_labels)

tensor_asn_enc = torch.Tensor(asn_enc)
le_asn = preprocessing.LabelEncoder()
tensor_sites_asn = le_asn.fit_transform(sites_asn)
tensor_sites_asn = torch.as_tensor(tensor_sites_asn)


In [36]:
import torch.utils.data as data

ip_dataset = data.TensorDataset(tensor_encodings,tensor_labels)
asn_dataset = data.TensorDataset(tensor_asn_enc,tensor_sites_asn)

In [37]:
batch_size = 64
validation_split = .2
shuffle_dataset = True
random_seed= 42

In [38]:
train_size = int(0.8 * len(asn_dataset))
test_size = len(asn_dataset) - train_size
asn_train_dataset, asn_test_dataset = torch.utils.data.random_split(asn_dataset, [train_size, test_size])
asn_train_loader = torch.utils.data.DataLoader(asn_train_dataset, batch_size=batch_size)
asn_test_loader = torch.utils.data.DataLoader(asn_test_dataset, batch_size=batch_size)
print(len(asn_train_loader.dataset), len(asn_test_loader.dataset))

16628 4158


In [39]:
train_size = int(0.8 * len(ip_dataset))
test_size = len(ip_dataset) - train_size
ip_train_dataset, ip_test_dataset = torch.utils.data.random_split(ip_dataset, [train_size, test_size])
ip_train_loader = torch.utils.data.DataLoader(ip_train_dataset, batch_size=batch_size)
ip_test_loader = torch.utils.data.DataLoader(ip_test_dataset, batch_size=batch_size)
print(len(ip_train_loader.dataset), len(ip_test_loader.dataset))

16628 4158


In [40]:
input_size_ipdataset = len(tensor_encodings[0])
output_size_ipdataset = len(site_date_ipset)
embed_size_ipdataset = 1000

input_size_asndataset = len(tensor_asn_enc[0])
output_size_asndataset = len(site_date_ipset)
embed_size_asndataset = 270
print(input_size_ipdataset, output_size_ipdataset)
print(input_size_asndataset, output_size_asndataset)

9760 193
346 193


In [41]:
model_ip = NNet(input_size_ipdataset, output_size_ipdataset)
model_asn = NNet(input_size_asndataset, output_size_asndataset)

In [42]:
criterion = nn.CrossEntropyLoss(reduction='mean')
optimizer_ip = torch.optim.SGD(model_ip.parameters(), lr=0.1)
optimizer_asn = torch.optim.SGD(model_asn.parameters(), lr=0.1)

In [43]:
# Reference: https://github.com/hunkim/PyTorchZeroToAll

from torch.autograd import Variable
import time
device = 'cpu'

def train(epoch, train_loader, model, optimizer):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} | Batch Status: {}/{} ({:.0f}%) | Loss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def test(test_loader, model):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        # sum up batch loss
        test_loss += criterion(output, target).item()
        # get the index of the max
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print(f'===========================\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} '
          f'({100. * correct / len(test_loader.dataset):.0f}%)')

In [44]:
# first ip dataset training and tests
since = time.time()
print(len(ip_train_loader.dataset), len(ip_test_loader.dataset))
for epoch in range(1, 10):
    epoch_start = time.time()
    train(epoch, ip_train_loader, model_ip, optimizer_ip)
    m, s = divmod(time.time() - epoch_start, 60)
    print(f'Training time: {m:.0f}m {s:.0f}s')
    test(ip_test_loader, model_ip)
    m, s = divmod(time.time() - epoch_start, 60)
    print(f'Testing time: {m:.0f}m {s:.0f}s')

m, s = divmod(time.time() - since, 60)
print(f'Total Time: {m:.0f}m {s:.0f}s\nModel was trained on {device}!')


16628 4158
Train Epoch: 1 | Batch Status: 0/16628 (0%) | Loss: 5.274475
Train Epoch: 1 | Batch Status: 640/16628 (4%) | Loss: 5.258980
Train Epoch: 1 | Batch Status: 1280/16628 (8%) | Loss: 5.271479
Train Epoch: 1 | Batch Status: 1920/16628 (12%) | Loss: 5.254810
Train Epoch: 1 | Batch Status: 2560/16628 (15%) | Loss: 5.246299
Train Epoch: 1 | Batch Status: 3200/16628 (19%) | Loss: 5.248940
Train Epoch: 1 | Batch Status: 3840/16628 (23%) | Loss: 5.255679
Train Epoch: 1 | Batch Status: 4480/16628 (27%) | Loss: 5.249626
Train Epoch: 1 | Batch Status: 5120/16628 (31%) | Loss: 5.214102
Train Epoch: 1 | Batch Status: 5760/16628 (35%) | Loss: 5.222743
Train Epoch: 1 | Batch Status: 6400/16628 (38%) | Loss: 5.228673
Train Epoch: 1 | Batch Status: 7040/16628 (42%) | Loss: 5.208768
Train Epoch: 1 | Batch Status: 7680/16628 (46%) | Loss: 5.215200
Train Epoch: 1 | Batch Status: 8320/16628 (50%) | Loss: 5.208424
Train Epoch: 1 | Batch Status: 8960/16628 (54%) | Loss: 5.217836
Train Epoch: 1 | Batc

Train Epoch: 5 | Batch Status: 8960/16628 (54%) | Loss: 2.408142
Train Epoch: 5 | Batch Status: 9600/16628 (58%) | Loss: 2.571864
Train Epoch: 5 | Batch Status: 10240/16628 (62%) | Loss: 2.505974
Train Epoch: 5 | Batch Status: 10880/16628 (65%) | Loss: 2.363601
Train Epoch: 5 | Batch Status: 11520/16628 (69%) | Loss: 2.224052
Train Epoch: 5 | Batch Status: 12160/16628 (73%) | Loss: 2.438471
Train Epoch: 5 | Batch Status: 12800/16628 (77%) | Loss: 1.979421
Train Epoch: 5 | Batch Status: 13440/16628 (81%) | Loss: 1.882197
Train Epoch: 5 | Batch Status: 14080/16628 (85%) | Loss: 2.142821
Train Epoch: 5 | Batch Status: 14720/16628 (88%) | Loss: 2.099058
Train Epoch: 5 | Batch Status: 15360/16628 (92%) | Loss: 1.821642
Train Epoch: 5 | Batch Status: 16000/16628 (96%) | Loss: 2.061512
Training time: 0m 9s
Test set: Average loss: 0.0328, Accuracy: 2013/4158 (48%)
Testing time: 0m 10s
Train Epoch: 6 | Batch Status: 0/16628 (0%) | Loss: 2.223071
Train Epoch: 6 | Batch Status: 640/16628 (4%) | L

In [45]:
# Now asn dataset training and tests
since = time.time()
criterion = nn.CrossEntropyLoss(reduction='mean')
print(len(asn_train_loader.dataset), len(asn_test_loader.dataset))
for epoch in range(1, 10):
    epoch_start = time.time()
    train(epoch, asn_train_loader, model_asn, optimizer_asn)
    m, s = divmod(time.time() - epoch_start, 60)
    print(f'Training time: {m:.0f}m {s:.0f}s')
    test(asn_test_loader, model_asn)
    m, s = divmod(time.time() - epoch_start, 60)
    print(f'Testing time: {m:.0f}m {s:.0f}s')

m, s = divmod(time.time() - since, 60)
print(f'Total Time: {m:.0f}m {s:.0f}s\nModel was trained on {device}!')

16628 4158
Train Epoch: 1 | Batch Status: 0/16628 (0%) | Loss: 5.256784
Train Epoch: 1 | Batch Status: 640/16628 (4%) | Loss: 5.257462
Train Epoch: 1 | Batch Status: 1280/16628 (8%) | Loss: 5.248040
Train Epoch: 1 | Batch Status: 1920/16628 (12%) | Loss: 5.250785
Train Epoch: 1 | Batch Status: 2560/16628 (15%) | Loss: 5.253596
Train Epoch: 1 | Batch Status: 3200/16628 (19%) | Loss: 5.244844
Train Epoch: 1 | Batch Status: 3840/16628 (23%) | Loss: 5.227427
Train Epoch: 1 | Batch Status: 4480/16628 (27%) | Loss: 5.220347
Train Epoch: 1 | Batch Status: 5120/16628 (31%) | Loss: 5.226476
Train Epoch: 1 | Batch Status: 5760/16628 (35%) | Loss: 5.233682
Train Epoch: 1 | Batch Status: 6400/16628 (38%) | Loss: 5.215895
Train Epoch: 1 | Batch Status: 7040/16628 (42%) | Loss: 5.190385
Train Epoch: 1 | Batch Status: 7680/16628 (46%) | Loss: 5.193106
Train Epoch: 1 | Batch Status: 8320/16628 (50%) | Loss: 5.160300
Train Epoch: 1 | Batch Status: 8960/16628 (54%) | Loss: 5.134292
Train Epoch: 1 | Batc

Train Epoch: 5 | Batch Status: 10880/16628 (65%) | Loss: 2.213195
Train Epoch: 5 | Batch Status: 11520/16628 (69%) | Loss: 2.251067
Train Epoch: 5 | Batch Status: 12160/16628 (73%) | Loss: 2.481195
Train Epoch: 5 | Batch Status: 12800/16628 (77%) | Loss: 2.325411
Train Epoch: 5 | Batch Status: 13440/16628 (81%) | Loss: 2.268681
Train Epoch: 5 | Batch Status: 14080/16628 (85%) | Loss: 2.222610
Train Epoch: 5 | Batch Status: 14720/16628 (88%) | Loss: 1.873897
Train Epoch: 5 | Batch Status: 15360/16628 (92%) | Loss: 2.387419
Train Epoch: 5 | Batch Status: 16000/16628 (96%) | Loss: 2.193258
Training time: 0m 1s
Test set: Average loss: 0.0341, Accuracy: 1802/4158 (43%)
Testing time: 0m 1s
Train Epoch: 6 | Batch Status: 0/16628 (0%) | Loss: 2.322102
Train Epoch: 6 | Batch Status: 640/16628 (4%) | Loss: 2.238785
Train Epoch: 6 | Batch Status: 1280/16628 (8%) | Loss: 2.044094
Train Epoch: 6 | Batch Status: 1920/16628 (12%) | Loss: 1.950035
Train Epoch: 6 | Batch Status: 2560/16628 (15%) | Loss